[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QWts4Kplj24mYyW8rlTL8Iizy1bwiggY?usp=sharing)

# SemEval-2018 Task 2: Emoji Prediction

This is the dataset called TweetEval [[1]](#section_id) and it is available [here](https://github.com/cardiffnlp/tweeteval) <br>

As described by [[1]](#section_id), the tweets were retrieved with the
Twitter API from October 2015 to February 2017 and ”geolocalized” in United States. <br>
This dataset includes tweets that contain one and only one emoji, of the 20 most frequent emojis.

In [1]:
import pandas as pd # Allow us to work with CSV files
import emoji # Allow us to print Emojis
import numpy as np # Allow us to work with arrays
import re  # Allow us to work with regular expressions
import nltk.data  # Allow to use the tokenizer punkt/english.pickle
import nltk # import the nltk package
from nltk.stem.snowball import SnowballStemmer # Import the SnowballStemmer algorithm
import warnings
warnings.filterwarnings('ignore') # Allow to disable Python warnings

# PENDIENTE
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression # Import logistic regression
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, f1_score, classification_report
# Import scikit-learn.metrics module for accuracy score, make_scorer, confusion matrix and classification_report
from sklearn import metrics # Import scikit-learn metrics module for Recall calculation
from sklearn.model_selection import cross_val_score # Import cross validation score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
# Import train_test_split function, stratified K-Folds cross-validator and GridSearchCV

### Step 1: Preparing the Emoji Prediction Mapping.
For this task, the **Emoji Prediction Mapping** proposed in the TweetEval [[1]](#section_id) will be used. <br>

In [2]:
# Create an Emoji Dictionary
emoji_dictionary = {
                    "0": "\u2764\uFE0F",    # :heart: prints a black heart instead of red heart
                    "1": "\U0001F60D",
                    "2": ":face_with_tears_of_joy:",
                    "3": ":two_hearts:",
                    "4": ":fire:",
                    "5": ":smiling_face_with_smiling_eyes:",
                    "6": ":smiling_face_with_sunglasses:",
                    "7": ":sparkles:",
                    "8": ":blue_heart: ",
                    "9": ":face_blowing_a_kiss:",
                    "10": ":camera:",
                    "11": ":United_States:",
                    "12": "\u2600\ufe0f",    # :sun: prints a black sun instead of a yellow sun
                    "13": ":purple_heart:",
                    "14": ":winking_face:",
                    "15": ":hundred_points:",
                    "16": ":beaming_face_with_smiling_eyes:",
                    "17": ":Christmas_tree:",
                    "18": ":camera_with_flash:",
                    "19": ":winking_face_with_tongue:"
                   }

# Create an Emoji list
emoji_list = []                          # Create a new-empty list to store all emojis obtained
for e in emoji_dictionary.values():      # For-loop iterates over all the dictionary values
    emoji_list.append(emoji.emojize(e))  # Add the emoji obtained of each iteration to the existing list "emoji_list" 

print (emoji_list) # Output: Emoji Prediction Mapping list.

# dictionary.values()       This function is used to display all the values of a dictionary. (Key is excluded)
# emoji.emojize             This function is used to transform the "unicode character" into its corresponding emoji.

['❤️', '😍', '😂', '💕', '🔥', '😊', '😎', '✨', '💙 ', '😘', '📷', '🇺🇸', '☀️', '💜', '😉', '💯', '😁', '🎄', '📸', '😜']


In [3]:
# Print the Emoji Prediction Mapping dataframe
df_emoji_mapping =  pd.DataFrame([emoji_list]) # Convert the emoji list into a Horizontal dataframe
display(df_emoji_mapping) # Output: Emoji Prediction Mapping dataframe

0  1  2  3  4  5  6  7   8  9  10  11  12 13 14 15 16 17 18 19
0  ❤️  😍  😂  💕  🔥  😊  😎  ✨  💙   😘  📷  🇺🇸  ☀️  💜  😉  💯  😁  🎄  📸  😜

The above dataframe can be explaned with the following examples: <br>
<br>
• As shown, 0 is equal to the red heart emoji (❤️), while 1 is equal to the heart eyes emoji (😍). <br>
&ensp;The same logic applies for the rest 18 emojis

### Step 2: Importing Emoji Prediction Train and Test subsets
For this task, the **Emoji Prediction train and test subset** proposed in the TweetEval [[1]](#section_id) will be used. <br>
Each dataset contains 40,000 tweets and represents the feature variable (X)

In [4]:
# Load the Emoji Prediction Train subset (Feature variable (X))

# Read a TXT file from internet (Github) that does not have a header
df_emoji_train_x = pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/'
                             'main/datasets/emoji/train_text.txt', header=None, sep ='\n', names=['Tweets'],
                              nrows = 40000)

print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
display(df_emoji_train_x) # Output: SemEval-2018 Emoji Prediction dataframe for training (truncated)



# Load the Emoji Prediction Test subset (Feature variable (X))

# Read a TXT file from internet (Github) that does not have a header
df_emoji_test_x = pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/'
                             'main/datasets/emoji/test_text.txt', header=None, sep ='\n', names=['Tweets'],
                              nrows = 40000)
    
print('\n\033[1mEmoji Prediction Test Subset:\033[0m')
display(df_emoji_test_x) # Output: SemEval-2018 Emoji Prediction dataframe for testing (truncated)


Emoji Prediction Train Subset:


Tweets
0      Sunday afternoon walking through Venice in the...
1      Time for some BBQ and whiskey libations. Chomp...
2      Love love love all these people ️ ️ ️ #friends...
3                                   ️ ️ ️ ️ @ Toys"R"Us 
4      Man these are the funniest kids ever!! That fa...
...                                                  ...
39995  Had a GREAT time at our Soul Sisters Holiday B...
39996                            ️ @ Hood River, Oregon 
39997  Recombinant fold? Something like that. Not a g...
39998  Gorgeous customer wearing our blue moon dress!...
39999  here's a clip of last month's heineken sponsor...

[40000 rows x 1 columns]


Emoji Prediction Test Subset:


Tweets
0                                      en Pelham Parkway
1      The calm before...... | w/ sofarsounds @user |...
2      Just witnessed the great solar eclipse @ Tampa...
3      This little lady is 26 weeks pregnant today! E...
4      Great road trip views! @ Shartlesville, Pennsy...
...                                                  ...
39995  I am another year older, wiser &amp; more grat...
39996    Back at it (@ Gen Korean BBQ in Carrollton, TX)
39997  The perfect place to spend your post-beach aft...
39998  Another beauty. ️#onlocationmakeup #indulgence...
39999  I definitely needed this ️#onewithnature @ Cas...

[40000 rows x 1 columns]

For this task, the **Emoji Prediction train and test subset (labels)**  proposed in the TweetEval [[1]](#section_id) will be used. <br>
Each dataset contains 40,000 tweets and represents the target variable (Y)

In [5]:
# Load the Emoji Prediction Train subset (Target Variable (Y))

# Read a TXT file from internet (Github) that does not have a header
df_emoji_train_y = pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/'
                             'main/datasets/emoji/train_labels.txt', header=None, names=['emoji label'],
                              nrows = 40000)

print('\n\033[1mEmoji Prediction Train Subset (Labels):\033[0m')
display(df_emoji_train_y) # Output: SemEval-2018 Emoji Prediction dataframe for training (labels)(truncated)



# Read a TXT file from internet (Github) that does not have a header
df_emoji_test_y = pd.read_csv('https://raw.githubusercontent.com/cardiffnlp/tweeteval/'
                             'main/datasets/emoji/test_labels.txt', header=None, names=['emoji label'],
                              nrows = 40000)

print('\n\033[1mEmoji Prediction Test Subset (Labels):\033[0m')
display(df_emoji_test_y) # Output: SemEval-2018 Emoji Prediction dataframe for testing (labels)(truncated)



Emoji Prediction Train Subset (Labels):


emoji label
0               12
1               19
2                0
3                0
4                2
...            ...
39995            0
39996            0
39997            0
39998            5
39999            1

[40000 rows x 1 columns]


Emoji Prediction Test Subset (Labels):


emoji label
0                2
1               10
2                6
3                1
4               16
...            ...
39995            0
39996            6
39997            2
39998            1
39999            0

[40000 rows x 1 columns]

In [6]:
# Merging the X and Y dataframes (Training)
df_emoji_train = pd.concat([df_emoji_train_x, df_emoji_train_y], axis=1)  # Merging the dataframes
print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
display(df_emoji_train) # Output: SemEval-2018 Emoji Prediction dataframe for training (truncated)


# Merging the X and Y dataframes (Testing)
df_emoji_test = pd.concat([df_emoji_test_x, df_emoji_test_y], axis=1)  # Merging the dataframes
print('\n\033[1mEmoji Prediction Test Subset:\033[0m')
display(df_emoji_test) # Output: SemEval-2018 Emoji Prediction dataframe for testing (truncated)


# .concact()      This function is used to concatenate two different dataframes.
# Axis=1          This parameter indicates column-wise concatenation (Merging columns of two different dataframes)


Emoji Prediction Train Subset:


Tweets  emoji label
0      Sunday afternoon walking through Venice in the...           12
1      Time for some BBQ and whiskey libations. Chomp...           19
2      Love love love all these people ️ ️ ️ #friends...            0
3                                   ️ ️ ️ ️ @ Toys"R"Us             0
4      Man these are the funniest kids ever!! That fa...            2
...                                                  ...          ...
39995  Had a GREAT time at our Soul Sisters Holiday B...            0
39996                            ️ @ Hood River, Oregon             0
39997  Recombinant fold? Something like that. Not a g...            0
39998  Gorgeous customer wearing our blue moon dress!...            5
39999  here's a clip of last month's heineken sponsor...            1

[40000 rows x 2 columns]


Emoji Prediction Test Subset:


Tweets  emoji label
0                                      en Pelham Parkway            2
1      The calm before...... | w/ sofarsounds @user |...           10
2      Just witnessed the great solar eclipse @ Tampa...            6
3      This little lady is 26 weeks pregnant today! E...            1
4      Great road trip views! @ Shartlesville, Pennsy...           16
...                                                  ...          ...
39995  I am another year older, wiser &amp; more grat...            0
39996    Back at it (@ Gen Korean BBQ in Carrollton, TX)            6
39997  The perfect place to spend your post-beach aft...            2
39998  Another beauty. ️#onlocationmakeup #indulgence...            1
39999  I definitely needed this ️#onewithnature @ Cas...            0

[40000 rows x 2 columns]

In [7]:
# Creating a function to print the corresponding emoji
def applyemoji(e):
    if e == 0:
        return '\u2764\uFE0F' # Print the red heart emoji
    elif e == 1:
        return '\U0001F60D'   # Print the heart eyes emoji
    elif e == 2:
        return '\U0001F602'   # Print the face with tears of joy emoji
    elif e == 3:
        return '\U0001F495'   # Print the two hearts emoji
    elif e == 4:
        return '\U0001F525'   # Print the fire emoji
    elif e == 5:
        return '\U0001F60A'   # Print the smiling face with smiling eyes emoji
    elif e == 6:
        return '\U0001F60E'   # Print the smiling face with sunglasses emoji
    elif e == 7:
        return '\u2728'       # Print the sparkles emoji
    elif e == 8:
        return '\U0001F499'   # Print the blue heart emoji
    elif e == 9:
        return '\U0001F618'   # Print the face blowing a kiss emoji
    elif e == 10:
        return '\U0001F4F7'   # Print the camera emoji
    elif e == 11:
        return '\U0001F1FA\U0001F1F8' # Print the US flag emoji  
    elif e == 12:
        return '\u2600\ufe0f' # Print the yellow sun emoji
    elif e == 13:
        return '\U0001F49C'   # Print the purple heart emoji
    elif e == 14:
        return '\U0001F609'   # Print the winking face emoji
    elif e == 15:
        return '\U0001F4AF'   # Print the hundred points emoji
    elif e == 16:
        return '\U0001F601'   # Print the beaming face with smiling eyes emoji
    elif e == 17:
        return '\U0001F384'   # Print the christmas tree emoji
    elif e == 18: 
        return '\U0001F4F8'   # Print the camera with flash emoji
    elif e == 19:
        return '\U0001F61C'   # Print the winking face with tongue emoji

In [8]:
# Checking the Emoji Prediction train subset after applied the function "applyemoji"
df_emoji_train['Emoji'] = df_emoji_train['emoji label'].apply(applyemoji) # Run the function "applyemoji"
print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
display(df_emoji_train[0:10])# Output: First 10 rows of the Emoji Prediction train subset


# Checking the Emoji Prediction test subset after applied the function "applyemoji"
df_emoji_test['Emoji'] = df_emoji_test['emoji label'].apply(applyemoji) # Run the function "applyemoji"
print('\n\033[1mEmoji Prediction Test Subset:\033[0m')
display(df_emoji_test[0:10]) # Output: First 10 rows of the Emoji Prediction test subset

# df.apply()   This command is used to pass a function and apply it on every single value of the column or dataframe.


Emoji Prediction Train Subset:


Tweets  emoji label Emoji
0  Sunday afternoon walking through Venice in the...           12    ☀️
1  Time for some BBQ and whiskey libations. Chomp...           19     😜
2  Love love love all these people ️ ️ ️ #friends...            0    ❤️
3                               ️ ️ ️ ️ @ Toys"R"Us             0    ❤️
4  Man these are the funniest kids ever!! That fa...            2     😂
5                 #sandiego @ San Diego, California            11    🇺🇸
6  My little ️ ️ ️ ️ ️ #ObsessedWithMyDog @ Cafe ...            0    ❤️
7  More #tinyepic things #tinyepicwestern, this o...           19     😜
8  Last night ️ @ Omnia Night Club At Caesars Pal...            0    ❤️
9  friendship at its finest. ....#pixar #toystory...            7     ✨


Emoji Prediction Test Subset:


Tweets  emoji label Emoji
0                                  en Pelham Parkway            2     😂
1  The calm before...... | w/ sofarsounds @user |...           10     📷
2  Just witnessed the great solar eclipse @ Tampa...            6     😎
3  This little lady is 26 weeks pregnant today! E...            1     😍
4  Great road trip views! @ Shartlesville, Pennsy...           16     😁
5  CHRISTMAS DEALS BUY ANY 3 SMALL POMADES 1.5 OR...           17     🎄
6  the #sisterstunt was mad real last night #MiaS...            4     🔥
7  I'm starting to love shooting in the dark #bra...           10     📷
8  Let the sun shine through ️ 5x5 Feet #oilpaint...           12    ☀️
9  Still bitch im trill never been no fiend :@vib...           18     📸

In [9]:
# Exploring the first 5 rows of the Emoji Prediction Train subset

# For-loop-enumerate iterates over indices (idx) and the first 5 rows (i) of a dataframe containing all the tweets (train subset)
for idx, i in enumerate(range(5)):      
    print(idx, '\n', df_emoji_train['Tweets'][i],
          emoji.emojize(emoji_dictionary[str(df_emoji_train['emoji label'][i])]),'\n')
# Output: First 5 tweets with their corresponding emoji used.

0 
 Sunday afternoon walking through Venice in the sun with @user ️ ️ ️ @ Abbot Kinney, Venice  ☀️ 

1 
 Time for some BBQ and whiskey libations. Chomp, belch, chomp! (@ Lucille's Smokehouse Bar-B-Que)  😜 

2 
 Love love love all these people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ San…  ❤️ 

3 
 ️ ️ ️ ️ @ Toys"R"Us  ❤️ 

4 
 Man these are the funniest kids ever!! That face! #HappyBirthdayBubb @ FLIPnOUT Xtreme  😂 



### Step 3: Preprocessing Emoji Prediction Train and Test subset

#### Cleaning Data (Part 1)
The first preprocessing phase will consist in the following actions:

|Action|Examples of the strings that will be removed or modified|
|:--|:-------------------------------|
|Lowercase the column "Tweets" | Can --> can, Texans --> texans, MLB --> mlb, Carly --> carly|
|Remove Stopwords |'a', 'about', 'above', 'after', 'again' .... "you're", "you've", 'your', 'yours', 'yourself', 'yourselves''|

1. In total, there are 179 stopwords in the NLTK module (stopwords.words('english')
2. However, 326 stopwords were added to the list. 
3. In total, there are 505 stopwords in this project

In [10]:
# Lowercasing the column "Tweets" (Emoji Prediction Train and Test subset) 

df_emoji_train_c1 = df_emoji_train.copy()                        # Create a copy of the Emoji Prediction Train subset
df_emoji_test_c1 = df_emoji_test.copy()                          # Create a copy of the Emoji Prediction Test subset

df_emoji_train["Tweets"] = df_emoji_train["Tweets"].str.lower()  # Lowercase the whole content of the column "Tweets" (Train)
df_emoji_test["Tweets"] = df_emoji_test["Tweets"].str.lower()    # Lowercase the whole content of the column "Tweets" (Test)

# Defining the Stopwords

stopwords = ["a's", "a", "about", "above", "according", "accordingly", "across", "actually", "after", "afterwards", "again", "against",
 "ain't", "ain", "all", "allow", "allows", "almost", "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anybody", "anyhow", "anyone", "anything", "anyway", "anyways", "anywhere", 
 "apart", "appear", "appropriate", "are", "aren't", "aren" "around", "as", "aside", "ask", "asking", "associated", "at", 
 "available", "be", "because", "been", "before", "beforehand", "behind", "being", "believe", "below", "beside", "besides", 
 "between", "beyond", "both", "brief", "but", "by", "c'mon", "c's", "came", "can", "can't", "cannot", "cant", "cause", 
 "causes", "certain", "certainly", "clearly", "co", "com", "come", "comes", "concerning", "consequently", "consider", 
 "considering", "contain", "containing", "contains", "corresponding", "could", "couldn't", "course", "currently", 
 "definitely", "described", "despite", "did", "didn't", "different", "do", "does", "doesn't", "doing", "don't", "done", 
 "down", "downwards", "during", "each", "edu", "eg", "either", "eight", "else", "elsewhere", "enough", "entirely", 
 "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "exactly", 
 "example", "far", "few", "fifth", "first", "five", "followed", "following", "follows", "for", "former", "formerly", 
 "forth", "four", "from", "further", "furthermore", "get", "gets", "getting", "given", "go", "goes", "going", "gone", 
 "got", "gotten", "had", "hadn't", "happens", "hardly", "has", "hasn't", "have", "haven't", "having", "he", "he's", 
 "hence", "her", "here", "here's", "hereafter","hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", 
 "hither", "hopefully", "how", "howbeit", "however", "i'd", "i'll", "i'm", "i've", "ie", "if", "immediate", "in", 
 "inasmuch", "inc", "indeed", "indicate", "indicated", "indicates", "inner", "insofar", "instead", "into", "inward", "is",
 "isn't", "it", "it'd", "it'll", "it's", "its", "itself", "just", "keep", "keeps", "kept", "know", "known", "knows", 
 "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "let's", "likely", "little", "look", 
 "looking", "looks", "ltd", "mainly", "many", "may", "maybe", "me", "meanwhile", "merely", "might", "more" , "moreover", 
 "most", "mostly", "much", "must", "my", "myself", "name", "namely", "nd", "near", "nearly", "necessary", "need", "needs",
 "neither", "never", "nevertheless","next", "nine", "no", "nobody", "non", "none", "noone", "nor", "normally", "not",
 "nothing", "novel", "now", "nowhere", "obviously", "of", "off", "often", "oh", "ok", "okay", "on", "once", "one", 
 "ones", "only", "onto", "or", "other", "others", "otherwise", "ought", "our", "ours", "ourselves", "out", "outside", 
 "over", "overall", "own", "particular", "particularly", "per", "perhaps", "placed", "plus", "possible", "presumably", 
 "probably", "provides", "que", "quite", "qv", "rather", "rd", "re", "really", "reasonably", "regarding", "regardless", 
 "regards", "relatively", "respectively", "right", "said", "same", "saw", "say", "saying", "says", "second", "secondly", 
 "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seriously", "seven", "several", 
 "shall", "she", "should", "shouldn't", "since", "six", "so", "some", "somebody", "somehow", "someone", "something", 
 "sometime", "sometimes", "somewhat", "somewhere", "soon", "specified", "specify", "specifying", "still", "sub", "such", 
 "sup", "t's", "take", "taken", "tell", "tends", "th", "than","that", "that's" , "thats", "the", "their", "theirs", "them", 
 "themselves", "then", "thence", "there", "there's", "thereafter", "thereby", "therefore", "therein", "theres", "thereupon",
 "these", "they", "they'd", "they'll", "they're", "they've", "think", "third", "this", "thorough", "thoroughly", "those", 
 "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "took", "toward", "towards", "tried", 
 "tries", "truly", "try", "trying", "twice", "two", "un", "under" , "unless", "unlikely", "until", "unto", "up", "upon", 
 "use", "used", "useful", "uses", "using", "usually", "value", "various", "very", "via", "viz", "vs", "want", "wants", 
 "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't" , "what", "what's", "whatever", "when", 
 "whence", "whenever", "where", "where's", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", 
 "whether", "which", "while", "whither", "who", "who's", "whoever", "whole", "whom", "whose", "why", "will", "willing", 
 "with", "within", "without", "won't", "wonder", "would", "wouldn't", "yes", "yet", "you", "you'd", "you'll", "you're", 
 "you've", "your", "yours", "yourself", "yourselves", "zero", "'s'"]


# .copy()          This function is used to make a copy of one dataframe with indices and data
# df.str.lower()   This function is used to transform the content of one column or dataframe to lowercase

In [11]:
# Removing Stopwords from the Emoji Prediction Train subset

for i in stopwords:         # For-loop iterates over all the words found in the list "stopwords"
    df_emoji_train['Tweets'] = df_emoji_train['Tweets'].replace(to_replace=r"\b%s\b"%(i), value='', regex=True)
# Action: Replace the stopwords by an empty character ('') (train)


# Removing Stopwords from the Emoji Prediction Test subset

for i in stopwords:         # For-loop iterates over all the words found in the list "stopwords"
    df_emoji_test['Tweets'] = df_emoji_test['Tweets'].replace(to_replace=r"\b%s\b"%(i), value='', regex=True)
# Action: Replace the stopwords by an empty character ('') (test)


# df.replace()   This function is used to replace occurrences of a particular sub-string with another sub-string.
#                In this case, the ReGex %s has been replaced by an empty character ('')
# To_replace     This parameter indicates the sub-string to replace
# value          This parameter indicates the sub-string to replace with
# ReGex=True     This parameter indicates that the sub-string to replace is a Regular Expression

# ReGex Explanation
# \bstring\b    This RegEx matches only the string declared.
#               In this case, it matches the variable string character (%s), which contains any of the stopwords declared.
#               For example. it will match either 'about', where' or 'has'

In [12]:
# Comparing the original Tweet vs the Tweet after first preprocessing

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
# This option is used to print the entire Pandas dataframe (all rows, all columns & all content)

# Comparing the Original Tweet VS the Tweet after first preprocessing (train subset)
print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
display(df_emoji_train_c1[['Tweets']].iloc[0:3]) # Output: Original Tweet 
display(df_emoji_train[['Tweets']].iloc[0:3]) # Output: Tweet after preprocessing 


# Comparing the Original Tweet VS the Tweet after first preprocessing (Test subset)
print('\n\033[1mEmoji Prediction Test Subset:\033[0m')
display(df_emoji_test_c1[['Tweets']].iloc[0:3]) # Output: Original Tweet
display(df_emoji_test[['Tweets']].iloc[0:3]) # Output: Tweet after preprocessing


Emoji Prediction Train Subset:


Tweets
0        Sunday afternoon walking through Venice in the sun with @user ️ ️ ️ @ Abbot Kinney, Venice 
1  Time for some BBQ and whiskey libations. Chomp, belch, chomp! (@ Lucille's Smokehouse Bar-B-Que) 
2      Love love love all these people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ San…

Tweets
0            sunday afternoon walking  venice   sun  @user ️ ️ ️ @ abbot kinney, venice 
1   time   bbq  whiskey libations. chomp, belch, chomp! (@ lucille's smokehouse bar-b-) 
2  love love love   people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ san…


Emoji Prediction Test Subset:


Tweets
0                                                             en Pelham Parkway
1  The calm before...... | w/ sofarsounds @user | : B. Hall.......#sofarsounds…
2                       Just witnessed the great solar eclipse @ Tampa, Florida

Tweets
0                                                    en pelham parkway
1   calm ...... | w/ sofarsounds @user | : b. hall.......#sofarsounds…
2                      witnessed  great solar eclipse @ tampa, florida

As shown above, the first 2 preprocessing techniques has been applied succesfully.

#### Cleaning Data (Part 2)
The second preprocessing phase will consist in the following actions:

|Action|Examples of the strings that will be removed or modified|
|:--|:-------------------------------|
|Remove User Objects |@user, @user_1, @user-1, @paulina_100, @WiNer206|
|Remove Hashtags | #friends #bff #celebrate #sandiego #sundayfunday #ObsessedWithMyDog|
|Remove Non-ASCII Characters |랙바, 에이오, ᴬᴺᴼᵀᴴᴱᴿ	ᴰᴿᴵᴺᴷ	ᴴᴬᴾᴾᵞ, บมาแล, добройночи|
|Remove new line characters|\n|
|Remove punctuation marks |!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~|
|Remove Two or more spaces|&ensp;, &ensp;&ensp;, &ensp;&ensp;&ensp;, &ensp;&ensp;&ensp;&ensp;,|
|Remove 1 or more underscores|"____ "_____" "_____________"|
|Remove numerical characters|0007, 0, 12389, 50000|
|Stemming|walking --> walk, excited -->, excit, kids --> kid|

1. The **Punkt/english.pickle** Sentence Tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start a sentence. <br>
2. The Punkt Sentence Tokenizer is based on the publication by [Kiss, T. & Strunk, J., 2006. Unsupervised Multilingual Sentence Boundary Detection. Computational Linguistics, 32(4), pp. 485-525](https://direct.mit.edu/coli/article/32/4/485/1923/Unsupervised-Multilingual-Sentence-Boundary)
3. SnowballStemmer is a stemming algorithm used to remove morphological affixes from words, leaving only the word stem. 
4. SnowballStemmer is part of the NLTK libraries

In [13]:
# Creating a function to clean data

def split_sentences(text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') # Get the The Punkt Sentence Tokenizer. 
    list_df = []                             # Create a new-empty list to store all sentences after preprocessing
    mark_sentence = '***'.join(tokenizer.tokenize(text))  
    # Add a sentence delimitator (***) to identify the beginning of each sentence
    
    #mark_sentence = re.sub(r"([a-z])\1+", r'\1', mark_sentence)
    # ReGex that removes duplicate letters. Replace them by the word without duplications)
    
    mark_sentence = re.sub(r"([@#][\w_-]+)", '', mark_sentence) 
    # ReGex that remove User Objects. (Replace them by Nothing (''))
    
    mark_sentence = re.sub(r"([^\x00-\x7F]+)", '', mark_sentence) 
    # ReGex that removes Non-ASCII Characters. (Replace them by nothing (''))
    
    mark_sentence = re.sub(r"([\b_\b]{1,})", '', mark_sentence) 
    # ReGex that removes 1 or more underscores. (Replace them by nothing (''))
    
    mark_sentence = re.sub(r"([0-9])", '', mark_sentence) 
    # ReGex that removes numbers. (Replace them by nothing (''))
    
    mark_sentence = re.sub(r"(\n+)", '', mark_sentence)
    # ReGex that removes new line characters (Replace them by nothing ('')
    
    mark_sentence = re.sub(r"([^\w\s])", ' ', mark_sentence) 
    # ReGex that removes punctuaction marks (Replace them by space (' '))
    
    mark_sentence = re.sub(r"([\s]{2,})", ' ', mark_sentence)
    # ReGex that removes Two or more spaces (Replace them by space (' '))
    
    list_df  = mark_sentence.split('***')   # Perform the sentence segmentation
    return list_df

print('Function split_sentences has been succesfully created')


# ReGex Explanation:
# \s    This RegEx matches any whitespace character. In other words, it will find strings such as " ", "\r" or "\n"
# \w    This ReGex matches any alphamumeric character. In other words, it will find strings such as: "a", "julio", "100", julio100"
# ()    The parenthesis identify a group of characters formed by the combination of 1, 2 or more characters.
#       For example, ([@][\w_-]) represents 1 group that has an structure "@ + Alphabetic Characters + _ or -"
# []    The brackets [] identify a range of characters.
#       For example, [\w\s] represents 1 range of values from any alphanumeric character or any whitespace character.
# ^     The caret symbol (^) inside of a character set [] represents the characters NOT in the range [\w\s]. 
#       (Different to the range [\w\s])
# \n    This character represents the new line character
# +     The plus sign (+) declares that "\n" is compulsory and should appears at least once. (1)
# {2,}  This quantifier declares that "\s" is compulsory and should appears at least two times. (2)
# [0-9] This ReGex matches any numerical character (0-9). In other words, it will find numbers such as "007" or "0"
# \bstring\b    This RegEx matches only the string declared.
#               In this case, it matches the underscore (_)

# \\X00      This means 0 in Hexa-decimal connotation
# \\7F       This means 127 in Hexa-decimal connotation.
# \x00-\x7F  This means a range from 0 to 127, which represents the range of the ASCII characters.

Function split_sentences has been succesfully created


In [14]:
# Cleaning the data (Part 2)(Train subset)

# Run the function "split_sentences" on the Train subset 
df_emoji_train['clean_tweet'] = df_emoji_train['Tweets'].apply(split_sentences)
# Action: Create a column 'clean_tweet' that stores the Tweets after preprocessing (training)

df_emoji_train['clean_tweet'] = df_emoji_train['clean_tweet'].apply(lambda x: ','.join(map(str, x)))
# Action: Transform the list obtained after preprocessing into single strings. (training)

df_emoji_train['clean_tweet'] = df_emoji_train['clean_tweet'].str.strip()
# Action: Remove extra spaces at the beginning and the end of any cell. (training)


# Cleaning the data (Part 2)(test subset)

# Run the function "split_sentences" on the Test subset
df_emoji_test['clean_tweet'] = df_emoji_test['Tweets'].apply(split_sentences)
# Action: Create a column 'clean_tweet' that stores the Tweets after preprocessing (test)

df_emoji_test['clean_tweet'] = df_emoji_test['clean_tweet'].apply(lambda x: ','.join(map(str, x)))
# Action: Transform the list obtained after preprocessing into single strings. (test)

df_emoji_test['clean_tweet'] = df_emoji_test['clean_tweet'].str.strip()
# Action: Remove extra spaces at the beginning and the end of any cell. (test)

print ('Done')

# df.apply()    This command is used to pass a function and apply it on every single value of the column or dataframe.
# lamda()       This command is only useful when we want to define a function that will be used only once in our program.
# ','.join()    This function takes all items in an iterable and joins them into one string.
#               In this case, it will take any word and seperate it with ",". For example: ['irwin','arnstein','subject']
# map()         This function is used to replace each value in a column or dataframe with another value.
#               In this case, this function transformed the list created after preprocessing into single strings
# str.strip()   This function is used to remove spaces at the beginning and the end of the cells.

Done


In [15]:
# Stemming the data

stemmer = SnowballStemmer('english') # Create a Stemmer object for 'English' language

# Stemming the data (Train subset)
df_emoji_train['clean_tweet1'] = df_emoji_train['clean_tweet'].apply(lambda x: [stemmer.stem(word) for word in x.split()])
# Action: Stem every word found in every row (train)

df_emoji_train['clean_tweet1'] = df_emoji_train['clean_tweet1'].apply(lambda x: ' '.join(map(str, x)))
# Action: Transform the list obtained after preprocessing into single strings (train)

df_emoji_train = df_emoji_train.drop(columns=['clean_tweet'])
# Drop the column "clean_tweet" (unstemmed column)(train)



# Stemming the data (Test subset)
df_emoji_test['clean_tweet1'] = df_emoji_test['clean_tweet'].apply(lambda x: [stemmer.stem(word) for word in x.split()]) 
# Action: Stem every word found in every row (test)

df_emoji_test['clean_tweet1'] = df_emoji_test['clean_tweet1'].apply(lambda x: ' '.join(map(str, x)))
# Action: Transform the list obtained after preprocessing into single strings (test)

df_emoji_test = df_emoji_test.drop(columns=['clean_tweet']) # Get rid of the unstemmed column.
# Drop the column "clean_tweet" (unstemmed column)(test)

print ('Done')

# df.apply()    This command is used to pass a function and apply it on every single value of the column or dataframe.
# lamda()       This command is only useful when we want to define a function that will be used only once in our program.
# x.split()     This function is used to split a string into a list using a user specified separator.
#               In this case, the content of each row is divided into words using a whitespace separator (' ')
# ','.join()    This function takes all items in an iterable and joins them into one string.
#               In this case, it will take any word and seperate it with ",". For example: ['irwin','arnstein','subject']
# map()         This function is used to replace each value in a column or dataframe with another value.
#               In this case, this function transformed the list created after preprocessing into single strings
# drop(columns='')  This function is used to delete 1 specific column
# stem()       This function is used to execute the stemmer object, which in this case is the NLTK Snowball Stemmer

Done


In [16]:
# Comparing the original Tweet vs the Tweet after second preprocessing

pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
# This option is used to print the entire Pandas dataframe (all rows, all columns & all content)

# Train subset
print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
display(df_emoji_train[0:5]) # Output: First 5 tweets (train)

# Test subset
print('\n\033[1mEmoji Prediction Test Subset:\033[0m')
display(df_emoji_test[0:5]) # Output: First 5 tweets (test)

# Column 'Tweets' contains the Original tweets
# Column 'clean_tweet1' contains the Tweets after all preprocessing


Emoji Prediction Train Subset:


Tweets  \
0            sunday afternoon walking  venice   sun  @user ️ ️ ️ @ abbot kinney, venice    
1   time   bbq  whiskey libations. chomp, belch, chomp! (@ lucille's smokehouse bar-b-)    
2  love love love   people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ san…    
3                                                                   ️ ️ ️ ️ @ toys"r"us    
4                   man    funniest kids !!  face! #happybirthdaybubb @ flipnout xtreme    

   emoji label Emoji  \
0           12    ☀️   
1           19     😜   
2            0    ❤️   
3            0    ❤️   
4            2     😂   

                                                        clean_tweet1  
0                 sunday afternoon walk venic sun abbot kinney venic  
1  time bbq whiskey libat chomp belch chomp lucill s smokehous bar b  
2                                           love love love peopl san  
3                                                           toy r us  
4                              man funniest kid face flipnout xtreme


Emoji Prediction Test Subset:


Tweets  \
0                                                      en pelham parkway   
1     calm ...... | w/ sofarsounds @user | : b. hall.......#sofarsounds…   
2                        witnessed  great solar eclipse @ tampa, florida   
3    lady  26 weeks pregnant today! excited  baby cam  ! @ springfield,…   
4                   great road trip views! @ shartlesville, pennsylvania   

   emoji label Emoji                                         clean_tweet1  
0            2     😂                                    en pelham parkway  
1           10     📷                             calm w sofarsound b hall  
2            6     😎                 wit great solar eclips tampa florida  
3            1     😍  ladi week pregnant today excit babi cam springfield  
4           16     😁       great road trip view shartlesvill pennsylvania

In [17]:
# Creating the final Emoji Prediction subset after preprocessing

pd.reset_option('^display.', silent=True) # Reset the default Pandas Display (Truncated)

# Final emoji prediction Train subset
print('\n\033[1mEmoji Prediction Train Subset:\033[0m')
df_emoji_train_final = df_emoji_train[['clean_tweet1','emoji label']] 
# Create the final Emoji Prediction dataframe (train)

display(df_emoji_train_final) # Output: Emoji prediction train subset (cleaning)

# Final emoji prediction Test subset
print('\n\033[1mEmoji Prediction test Subset:\033[0m')
df_emoji_test_final = df_emoji_test[['clean_tweet1','emoji label']]
# Create the final Emoji Prediction dataframe (test)

display(df_emoji_test_final)  # Output: Emoji prediction test subset (cleaning)



Emoji Prediction Train Subset:


clean_tweet1  emoji label
0      sunday afternoon walk venic sun abbot kinney v...           12
1      time bbq whiskey libat chomp belch chomp lucil...           19
2                               love love love peopl san            0
3                                               toy r us            0
4                  man funniest kid face flipnout xtreme            2
...                                                  ...          ...
39995         great time soul sister holiday brunch love            0
39996                                  hood river oregon            0
39997  recombin fold like geologist geolog field trip...            0
39998               gorgeous custom wear blue moon dress            5
39999  s clip month s heineken sponsor parti tonight ...            1

[40000 rows x 2 columns]


Emoji Prediction test Subset:


clean_tweet1  emoji label
0                                      en pelham parkway            2
1                               calm w sofarsound b hall           10
2                   wit great solar eclips tampa florida            6
3      ladi week pregnant today excit babi cam spring...            1
4         great road trip view shartlesvill pennsylvania           16
...                                                  ...          ...
39995  i year older wiser amp grate pool dark harrah ...            0
39996                  back gen korean bbq carrollton tx            6
39997  perfect place spend post beach afternoon hour ...            2
39998                                             beauti            1
39999                            i need castl rock trail            0

[40000 rows x 2 columns]

In [19]:
# Transforming the Emoji Prediction subsets to Python lists.

# Train subset
list_emoji_train_final = df_emoji_train_final['clean_tweet1'].tolist()
# Action: Transform the Emoji Prediction Train dataframe into a Python list

# Test subset
list_emoji_test_final = df_emoji_test_final['clean_tweet1'].tolist()
# Action: Transform the Emoji Prediction Test dataframe into a Python list

display(list_emoji_train_final) # Output: Emoji Prediction Train list
#Uncomment the following line if you want to see the Emoji Prediction Test list
#display(list_emoji_test_final) 

# df.values.tolist()    This function is used to convert a dataFrame into a Python list

['sunday afternoon walk venic sun abbot kinney venic',
 'time bbq whiskey libat chomp belch chomp lucill s smokehous bar b',
 'love love love peopl san',
 'toy r us',
 'man funniest kid face flipnout xtreme',
 'san diego california',
 'cafe solstic capitol hill',
 'thing crazi i',
 'night omnia night club caesar palac',
 'friendship finest',
 'i l ve work yesterday s balloon decor snn th annual back school health',
 'dem shoe tho lol readi copleypriceymca',
 'i love weekend calebrancourt canneri row monterey bay ca',
 'yesss august full love birthday los angel',
 'i pic eminem perform vibe',
 'i l ve work yesterday s balloon decor snn th annual back school health',
 'sunday cute babi dog',
 'hot find shade roadraceengin booth',
 'kid skyspac los angel',
 'amaz night pretti amaz peopl pandora summer',
 'thank incred night night shrine auditorium amp expo hall',
 'thank amaz night pandora summer',
 'draw line',
 'i hate taylor visit hous',
 'thank boy perform well night i love pandora',


### Step 4: Performing Word Embeddings

Word embeddings is nothing but the process of converting text data to numerical vectors, and it is used to capture not only  the semantic of the word, but also their emotional content. <br>

In this case, the **Bag-of-Words** technique will be applied in this project.
1. BoW is a representation of text that describes the occurrence of words within a document collection.
2. BoW uses word occurrence frequencies to measure the content of the tweet and see how often each words appeared
3. BoW is a method to extract features (X) from tweets. These attributes should be used for training any Machine Learning algorithm.

In [20]:
# Vectorizing the Emoji Prediction Train subset

#Uncomment the following line if you want to print all columns and all content of a given dataframe. 
# (Truncated rows, all columns & all content)
#pd.set_option("display.max_columns", None, 'display.max_colwidth', None)

vect_train = CountVectorizer(dtype=np.uint8, max_features = 3000, min_df = 5)
Z_train = vect_train.fit_transform(list_emoji_train_final)
df_vect_train = pd.DataFrame(Z_train.A, columns=vect_train.get_feature_names())
display(df_vect_train)

print("--------------------------------------------------")
display(list_emoji_train_final[0])

ab  abbey  abbot  abl  absolut  academi  accessori  accident  \
0       0      0      1    0        0        0          0         0   
1       0      0      0    0        0        0          0         0   
2       0      0      0    0        0        0          0         0   
3       0      0      0    0        0        0          0         0   
4       0      0      0    0        0        0          0         0   
...    ..    ...    ...  ...      ...      ...        ...       ...   
39995   0      0      0    0        0        0          0         0   
39996   0      0      0    0        0        0          0         0   
39997   0      0      0    0        0        0          0         0   
39998   0      0      0    0        0        0          0         0   
39999   0      0      0    0        0        0          0         0   

       accomplish  ace  ...  yosemit  young  youth  youtub  yrs  yum  yummi  \
0               0    0  ...        0      0      0       0    0    0      0   
1               0    0  ...        0      0      0       0    0    0      0   
2               0    0  ...        0      0      0       0    0    0      0   
3               0    0  ...        0      0      0       0    0    0      0   
4               0    0  ...        0      0      0       0    0    0      0   
...           ...  ...  ...      ...    ...    ...     ...  ...  ...    ...   
39995           0    0  ...        0      0      0       0    0    0      0   
39996           0    0  ...        0      0      0       0    0    0      0   
39997           0    0  ...        0      0      0       0    0    0      0   
39998           0    0  ...        0      0      0       0    0    0      0   
39999           0    0  ...        0      0      0       0    0    0      0   

       yup  zone  zoo  
0        0     0    0  
1        0     0    0  
2        0     0    0  
3        0     0    0  
4        0     0    0  
...    ...   ...  ...  
39995    0     0    0  
39996    0     0    0  
39997    0     0    0  
39998    0     0    0  
39999    0     0    0  

[40000 rows x 3000 columns]

--------------------------------------------------


'sunday afternoon walk venic sun abbot kinney venic'

In [21]:
# Vectorizing the Test subset

pd.set_option("display.max_columns", None, 'display.max_colwidth', None)
vect_test= CountVectorizer(dtype=np.uint8, max_features = 3000, min_df = 5)
Z_test = vect_test.fit_transform(list_emoji_test_final)
df_vect_test = pd.DataFrame(Z_test.A, columns=vect_test.get_feature_names())
display(df_vect_test)

print("--------------------------------------------------")
display(list_emoji_test_final[0])

ab  abl  absolut  academi  accept  account  ace  acr  act  action  \
0       0    0        0        0       0        0    0    0    0       0   
1       0    0        0        0       0        0    0    0    0       0   
2       0    0        0        0       0        0    0    0    0       0   
3       0    0        0        0       0        0    0    0    0       0   
4       0    0        0        0       0        0    0    0    0       0   
...    ..  ...      ...      ...     ...      ...  ...  ...  ...     ...   
39995   0    0        0        0       0        0    0    0    0       0   
39996   0    0        0        0       0        0    0    0    0       0   
39997   0    0        0        0       0        0    0    0    0       0   
39998   0    0        0        0       0        0    0    0    0       0   
39999   0    0        0        0       0        0    0    0    0       0   

       activ  actual  ad  adam  add  addict  addit  adopt  ador  adult  \
0          0       0   0     0    0       0      0      0     0      0   
1          0       0   0     0    0       0      0      0     0      0   
2          0       0   0     0    0       0      0      0     0      0   
3          0       0   0     0    0       0      0      0     0      0   
4          0       0   0     0    0       0      0      0     0      0   
...      ...     ...  ..   ...  ...     ...    ...    ...   ...    ...   
39995      0       0   0     0    0       0      0      0     0      0   
39996      0       0   0     0    0       0      0      0     0      0   
39997      0       0   0     0    0       0      0      0     0      0   
39998      0       0   0     0    0       0      0      0     0      0   
39999      0       0   0     0    0       0      0      0     0      0   

       advanc  adventur  af  afraid  african  afternoon  age  ago  agre  ah  \
0           0         0   0       0        0          0    0    0     0   0   
1           0         0   0       0        0          0    0    0     0   0   
2           0         0   0       0        0          0    0    0     0   0   
3           0         0   0       0        0          0    0    0     0   0   
4           0         0   0       0        0          0    0    0     0   0   
...       ...       ...  ..     ...      ...        ...  ...  ...   ...  ..   
39995       0         0   0       0        0          0    0    0     0   0   
39996       0         0   0       0        0          0    0    0     0   0   
39997       0         0   0       0        0          1    0    0     0   0   
39998       0         0   0       0        0          0    0    0     0   0   
39999       0         0   0       0        0          0    0    0     0   0   

       ahead  aint  air  airlin  airport  aka  al  alabama  alamo  albani  \
0          0     0    0       0        0    0   0        0      0       0   
1          0     0    0       0        0    0   0        0      0       0   
2          0     0    0       0        0    0   0        0      0       0   
3          0     0    0       0        0    0   0        0      0       0   
4          0     0    0       0        0    0   0        0      0       0   
...      ...   ...  ...     ...      ...  ...  ..      ...    ...     ...   
39995      0     0    0       0        0    0   0        0      0       0   
39996      0     0    0       0        0    0   0        0      0       0   
39997      0     0    0       0        0    0   0        0      0       0   
39998      0     0    0       0        0    0   0        0      0       0   
39999      0     0    0       0        0    0   0        0      0       0   

       album  ale  alert  alex  alexandria  ali  aliv  alley  allow  alon  \
0          0    0      0     0           0    0     0      0      0     0   
1          0    0      0     0           0    0     0      0      0     0   
2          0    0      0     0           0    0     0      0      0     0   
3          0    0      

--------------------------------------------------


'en pelham parkway'

In [22]:
# Transfering the data into 2 numpy arrays

x_train = df_vect_train.loc[:,:].to_numpy() 
# This array contains all the feature variable values.

y_train = df_emoji_train_final.loc[:, df_emoji_train_final.columns == 'emoji label'].to_numpy()
# This array contains ONLY the target variable values.

print('\n'+'\033[1m'+'Array of feature variables (Train):'+'\033[0m', x_train.shape) 
print(x_train) # Output: All possible elements of the array. (This includes everything except the values of the target variable ("Class label")

print('\n'+'\033[1m'+'Array of target variable (Train):'+'\033[0m', y_train.shape)
print(y_train) # Output: All possible elements of the array. (This includes only values of "Class Label" feature from the wine dataset)


Array of feature variables (Train): (40000, 3000)
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Array of target variable (Train): (40000, 1)
[[12]
 [19]
 [ 0]
 ...
 [ 0]
 [ 5]
 [ 1]]


In [23]:
# Transfering the data into 2 numpy arrays

x_test = df_vect_test.loc[:,:].to_numpy() 
# This array contains all the feature variable values.

y_test = df_emoji_test_final.loc[:, df_emoji_test_final.columns == 'emoji label'].to_numpy()
# This array contains ONLY the target variable values.

print('\n'+'\033[1m'+'Array of feature variables (Test):'+'\033[0m', x_test.shape) 
print(x_test) # Output: All possible elements of the array. (This includes everything except the values of the target variable ("Class label")

print('\n'+'\033[1m'+'Array of target variable (Test):'+'\033[0m', y_test.shape)
print(y_test) # Output: All possible elements of the array. (This includes only values of "Class Label" feature from the wine dataset)


Array of feature variables (Test): (40000, 3000)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Array of target variable (Test): (40000, 1)
[[ 2]
 [10]
 [ 6]
 ...
 [ 2]
 [ 1]
 [ 0]]


### Step 5: Building the Machine Learning model

#### Naıve  Bayes  Multinomial  classifier

In [24]:
# Use a Naive Bayes model 
from sklearn.naive_bayes import MultinomialNB 

mnb = MultinomialNB() 

# Train the model
mnb.fit(x_train, y_train)

MultinomialNB()

In [25]:
# Take the model that was trained on the X_train_cv data and apply it to the X_test_cv data 
y_pred_cv_mnb = mnb.predict(x_test) 
y_pred_cv_mnb # The output is all of the predictions

array([17,  0,  8, ...,  8,  7,  2], dtype=int64)

In [26]:
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(y_test, y_pred_cv_mnb))
print(classification_report(y_test, y_pred_cv_mnb))

0.079225
              precision    recall  f1-score   support

           0       0.21      0.20      0.21      8618
           1       0.09      0.07      0.08      3829
           2       0.08      0.07      0.08      3646
           3       0.05      0.04      0.04      2093
           4       0.07      0.05      0.06      2987
           5       0.03      0.02      0.03      1304
           6       0.04      0.03      0.03      1604
           7       0.05      0.04      0.04      2170
           8       0.03      0.04      0.03      1266
           9       0.03      0.03      0.03       941
          10       0.03      0.04      0.04      1115
          11       0.04      0.06      0.05      1578
          12       0.03      0.03      0.03      1004
          13       0.02      0.03      0.02       892
          14       0.03      0.05      0.04      1013
          15       0.02      0.04      0.03       989
          16       0.02      0.04      0.03       918
          17      

#### Logistic Regression Classifier

In [27]:
# Create the Logistic Regression object
logr = LogisticRegression(solver='lbfgs')  
# Train the model
logr.fit(x_train, y_train)

LogisticRegression()

In [28]:
# Take the model that was trained on the X_train_cv data and apply it to the X_test_cv data 
y_pred_cv_logr = logr.predict(x_test) 
y_pred_cv_logr # The output is all of the predictions

array([0, 0, 1, ..., 8, 2, 2], dtype=int64)

In [29]:
print(accuracy_score(y_test, y_pred_cv_logr))
print(classification_report(y_test, y_pred_cv_logr))

0.1283
              precision    recall  f1-score   support

           0       0.22      0.40      0.28      8618
           1       0.10      0.17      0.12      3829
           2       0.09      0.15      0.11      3646
           3       0.05      0.03      0.04      2093
           4       0.07      0.04      0.05      2987
           5       0.03      0.02      0.02      1304
           6       0.03      0.02      0.03      1604
           7       0.05      0.04      0.04      2170
           8       0.03      0.01      0.02      1266
           9       0.02      0.01      0.01       941
          10       0.03      0.02      0.03      1115
          11       0.04      0.01      0.01      1578
          12       0.03      0.01      0.01      1004
          13       0.04      0.00      0.01       892
          14       0.02      0.01      0.01      1013
          15       0.01      0.00      0.00       989
          16       0.04      0.02      0.02       918
          17       0

In [ ]:
pd.set_option("display.max_columns", None, 'display.max_colwidth', None)

(df_emoji_train[df_emoji_train['clean_tweet1'].str.contains("sendfoodnotnud")])
#display(df_emoji_train_x[df_emoji_train_x['Tweets'].str.contains("")])

display(df_emoji_train_x[['Tweets']].iloc[120:150]) 

# Comparing one paper in dataframes df_covidp and metadata_df.
#display(df_emoji_train.loc[df_emoji_train['Tweets'] == 'dancing jelly beanss'])
# Filter all rows containing the string in the specified column
#display(df_emoji_train.loc[df_emoji_train['clean_tweet1'] == 'dancing jelly beanss'])

In [30]:
df_emoji_pred = 

0 
 sunday afternoon walking  venice   sun  @user ️ ️ ️ @ abbot kinney, venice  ☀️ 

1 
 time   bbq  whiskey libations. chomp, belch, chomp! (@ lucille's smokehouse bar-b-)  😜 

2 
 love love love   people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ san…  ❤️ 

3 
 ️ ️ ️ ️ @ toys"r"us  ❤️ 

4 
 man    funniest kids !!  face! #happybirthdaybubb @ flipnout xtreme  😂 



In [ ]:
X_train_tokens = vect_test.get_feature_names()
len(X_train_tokens)

In [ ]:
# number of times each token appears across all HAM messages
token_0 = mnb.feature_count_[0, :]
token_1 = mnb.feature_count_[1, :]
token_2 = mnb.feature_count_[2, :]
token_3 = mnb.feature_count_[3, :]
token_4 = mnb.feature_count_[4, :]
token_5 = mnb.feature_count_[5, :]
token_6 = mnb.feature_count_[6, :]
token_7 = mnb.feature_count_[7, :]
token_8 = mnb.feature_count_[8, :]
token_9 = mnb.feature_count_[9, :]
token_10 = mnb.feature_count_[10, :]
token_11 = mnb.feature_count_[11, :]
token_12= mnb.feature_count_[12, :]
token_13 = mnb.feature_count_[13, :]
token_14 = mnb.feature_count_[14, :]
token_15 = mnb.feature_count_[15, :]
token_16 = mnb.feature_count_[16, :]
token_17 = mnb.feature_count_[17, :]
token_18 = mnb.feature_count_[18, :]
token_19 = mnb.feature_count_[19, :]

In [ ]:
tokens = pd.DataFrame({'token':X_train_tokens, 
                       '0':token_0, 
                       '1':token_1,
                       '2':token_2,
                       '3':token_3,
                       '4':token_4,
                       '5':token_5,
                       '6':token_6,
                       '7':token_7,
                       '8':token_8,
                       '9':token_9,
                       '10':token_10,
                       '11':token_11,
                       '12':token_12,
                       '13':token_13,
                       '14':token_14,
                       '15':token_15,
                       '16':token_16,
                       '17':token_17,
                       '18':token_18,
                       '19':token_19,}).set_index('token')
tokens.head()


In [ ]:
# convert the ham and spam counts into frequencies
tokens['0'] = tokens['0'] / mnb.class_count_[0]
#tokens['1'] = tokens.1 / nb.class_count_[1]
tokens.sort_values(by=['0'], ascending = False)

In [ ]:
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
neg_class_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]


print(np.take(vect_train.get_feature_names(), neg_class_prob_sorted[:10]))
print(np.take(vect_train.get_feature_names(), pos_class_prob_sorted[:10]))

In [ ]:
import emoji
print(emoji.emojize('Phd is very easy!!! :Namibia:'))
print('\U0001F606')
# ! pip install emoji

In [ ]:
for i in stopwords:
    df_emoji_train['Tweetsts'] = test.replace(to_replace=r'\b%s\b'%i, value="",regex=True)

In [ ]:
name = 'julio'
print("My name is %s" %(name))

## References

<a id='section_id'></a>
&emsp;[1] F. Barbieri, J. Camacho-Collados, F. Ronzano, L. Espinosa-Anke, M. Ballesteros, V. Basile, V. Patti and H. Saggion, "Semeval 2018 Task 2: Multilingual<br>
&emsp;&emsp;&ensp;Emoji Prediction," in proceedings of the 12th International Workshop on Semantic Evaluation, 2018, pp. 24-33<br>
